
#### This is a Kaggle Jupyter Notebook. 
#### This script connects to the API Football data to fetch the Premier League standings results in a json file (Extract). 
#### Next the data is arranged into a list of tuples which can be parsed into a pandas dataframe (Transform). 
#### Finally the script connects to a database in Azure Database for MySQL to update a table with the standings results.


In [59]:
# load all necessary libraries

!pip install mysql-connector-python # install mysql-connector-python package to connect with MySQL 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json
import requests
import mysql.connector
from kaggle_secrets import UserSecretsClient # enable kaggle secrets for this notebook to replace .env file


In [60]:
# retrieve credentials to connect API

API_KEY = UserSecretsClient().get_secret("API_KEY")
API_HOST = UserSecretsClient().get_secret("API_HOST")

url = API_HOST
headers = {"X-Auth-Token": API_KEY}

**EXTRACT DATA FROM API -  -----------------------------------------------------------**

In [61]:
# fetch data from API

response = requests.get(url, headers = headers).json()


In [62]:
# filter the response down to the standings table list

standings_list = response["standings"][0]["table"]


**TRANSFORM----------------------------------------------------------------------------------------------------**

In [63]:
# arrange clubs records into a list of tuples

rows = [] 

for club in standings_list:
    season = 2025
    position = club["position"]
    team_id = club["team"]["id"]
    team = club["team"]["name"]
    played = club["playedGames"]
    won = club["won"]
    draw = club["draw"]
    lost = club["lost"]
    goals_for = club["goalsFor"]
    goals_against = club["goalsAgainst"]
    goal_diff = club["goalDifference"]
    points = club["points"]
    form = club["form"]
    
    club_records = (season, position, team_id, team, played, won, draw, lost, goals_for, goals_against, goal_diff, points, form)
    rows.append(club_records)


In [64]:
# create pandas DataFrame

column_names = ["season", "position", "team_id", "team", "played", "won", "draw", "lost", "goals_for", "goals_against", "goal_diff", "points", "form"]

standings_df = pd.DataFrame(rows, columns = column_names)
standings_df.head()

,season,position,team_id,team,played,won,draw,lost,goals_for,goals_against,goal_diff,points,form
0,2025,1,64,Liverpool FC,3,3,0,0,8,4,4,9,None
1,2025,2,61,Chelsea FC,3,2,1,0,7,1,6,7,None
2,2025,3,57,Arsenal FC,3,2,0,1,6,1,5,6,None
3,2025,4,73,Tottenham Hotspur FC,3,2,0,1,5,1,4,6,None
4,2025,5,62,Everton FC,3,2,0,1,5,3,2,6,None


**LOAD-------------------------------------------------------------------------------------------------------**

In [65]:
# retrieve credentials to connect to MySQL

MYSQL_HOST = UserSecretsClient().get_secret("MYSQL_HOST")
MYSQL_PORT = UserSecretsClient().get_secret("MYSQL_PORT")
MYSQL_USER = UserSecretsClient().get_secret("MYSQL_USER")
MYSQL_PWD = UserSecretsClient().get_secret("MYSQL_PWD")
MYSQL_DB = UserSecretsClient().get_secret("MYSQL_DB")
MYSQL_DB

'premier_league_db'

In [66]:
# test connection to MySQL server

server_conn = mysql.connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PWD,
    ssl_ca = "/kaggle/input/ca-cert/DigiCertGlobalRootG2.crt.pem", ssl_disabled = False,
    connection_timeout = 10, # process will stops after 10asec without a response
    raise_on_warnings = True # turns MySQL errors into python exceptions
)
server_cur = server_conn.cursor()
print(f"[SUCCESS] - Connected to MySQL server!")

# close connection to server
server_cur.close()
server_conn.close()

[SUCCESS] - Connected to MySQL server!


In [82]:
# connect to premier_league_db database

db_conn = mysql.connector.connect(
    host = MYSQL_HOST,
    port = MYSQL_PORT,
    user = MYSQL_USER,
    password = MYSQL_PWD,
    database = MYSQL_DB,
    ssl_ca = "/kaggle/input/ca-cert/DigiCertGlobalRootG2.crt.pem", ssl_disabled = False,
    connection_timeout = 10, # process will stops after 10asec without a response
    autocommit = False, # transactions are written in the database ONLY when we call a commit operation
    raise_on_warnings = True # turns MySQL errors into python exceptions
)
cursor = db_conn.cursor()
print(f"Connection to database successful!")

Connection to database successful!


In [68]:
# test if table exists in database

sql_table = "standings"
cursor.execute("SHOW TABLES LIKE %s", (f"{sql_table}",))

if cursor.fetchone() is None:
    raise SystemExit(f"This table '{sql_table}' does not exist...please create it")
else:
    print(f"[SUCCESS] - The table '{sql_table}' exists... continue to next step")

[SUCCESS] - The table 'standings' exists... continue to next step


In [83]:
# number of rows in table

no_rows = len(rows)

# update columns with variable values

UPSERT_SQL = (f"""
INSERT INTO {sql_table}
(season, position, team_id, team, played, won, draw, lost, goals_for, goals_against, goal_diff, points, form)
VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) AS src
ON DUPLICATE KEY UPDATE
position = src.position,
team = src.team,
played = src.played,
won = src.won,
draw = src.draw,
lost = src.lost,
goals_for = src.goals_for,
goals_against = src.goals_against,
goal_diff = src.goal_diff,
points = src.points,
form = src.form;"""
)


In [84]:
# run UPSERT operation in MySQL

try:
    cursor.executemany(UPSERT_SQL, rows)
    db_conn.commit()
    print(f"[SUCCESS] - Upsert for '{no_rows}' rows")
except Exception as e:
    db_conn.rollback()
    print(f"[ERROR] - Rolled back due to the following error: {e}")
finally:
    cursor.close()
    db_conn.close()
    print("\nAll database connections closed. \nClean up completed.")

[SUCCESS] - Upsert attemtepted for '20' rows

All database connections closed. 
Clean up completed.
